### «Добавление веб-поиска в агента»

**Цель**: сравнить поведение агента без инструментов и с инструментом веб-поиска, чтобы показать, как агент получает доступ к свежей информации.

In [5]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")

llm = ChatOpenAI(
    model="google/gemini-2.5-flash",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

### Часть 1: Агент без внешних источников

In [7]:
agent = create_agent(model=llm)

question = HumanMessage(content="Кто является действующим мэром Сан-Франциско?")

response = agent.invoke(
    {"messages": [question]}
)
print(response['messages'][-1].content)

Действующим мэром Сан-Франциско является **Лондон Брид**. 

Она занимает эту должность с июля 2018 года.


### Часть 2: Создание инструмента веб-поиска

#### Что происходит:
1. TavilyClient — это клиент для Tavily API, одного из лучших search API для LLM (оптимизирован под агентов, возвращает чистые результаты).
2. @tool превращает функцию web_search в инструмент, понятный LangChain.
3. Описание (docstring) говорит модели: «Этот инструмент ищет в интернете».
4. Тестовый вызов — проверяем, что инструмент работает сам по себе.
#### Почему Tavily?
- Возвращает структурированный JSON с results, answer, query.
- Фильтрует рекламу и мусор.
- Поддерживает deep search (платный), но даже бесплатный режим отлично работает для агентов.

In [6]:
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:
    """Search the web information"""
    return tavily_client.search(query)

web_search.invoke("Кто является действующим мэром Сан-Франциско?")

{'query': 'Кто является действующим мэром Сан-Франциско?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://stmegi.com/posts/121779/merom-san-frantsisko-izbran-deniel-lure/',
   'title': 'Мэром Сан-Франциско избран Дэниел Лурье - STMEGI',
   'content': 'Лурье, демократ, является наследником состояния компании Levi Strauss. Он сын раввина Брайана Лурье, который занимал пост исполнительного',
   'score': 0.9998697,
   'raw_content': None},
  {'url': 'https://rossaprimavera.ru/news/72bd8f9b',
   'title': "Мэром Сан-Франциско избран наследник компании Levi's",
   'content': 'Действующий мэр Сан-Франциско Лондон Брид уже признала поражение и поздравила Лури с победой. Лури станет первым мэром Сан-Франциско за последние 100 лет, который никогда не работал в правительстве до своего избрания. Лури является умеренным демократом и обещал избирателям расширить штат полиции, построить больше мест в приютах для бездомных, закрыть места торговли наркотиками

### Часть 3: Агент с веб-поиском

In [9]:
agent = create_agent(
    model=llm,
    tools=[web_search]
)

question = HumanMessage(content="Кто является действующим мэром Сан-Франциско?")
response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

Действующим мэром Сан-Франциско является Дэниел Лурье. Он стал 46-м мэром города и округа Сан-Франциско. Ранее этот пост занимала Лондон Брид.
